## 日志的监控
在平常，我们经常会遇到需要监控服务日志的输出情况，实时监测里面是否有FATAL、WARN日志或者其他异常。在shell里面我们可以使用```tail -f```或者```tail -F```(日志rotate)。在python里面，我们可以使用pyinotify库来监控日志

## pyinotify
Pyinotify是一个Python模块，用来监测文件系统的变化。 Pyinotify依赖于Linux内核的功能—inotify（内核2.6.13合并）。inotify的是一个事件驱动的通知器，其通知接口通过三个系统调用从内核空间到用户空间。pyinotify结合这些系统调用，并提供一个顶级的抽象和一个通用的方式来处理这些功能。
inotify监测的常用文件系统事件如下表:

|     事件名称     |                           说明                           |
| ---------------- | -------------------------------------------------------- |
| IN_ACCESS        | 文件被访问                                               |
| IN_MODIFY        | 文件被修改                                               |
| IN_ATTRIB        | 文件属性发生变化, 如chmod、chown、touch等                |
| IN_CLOSE_WRITE   | 可写文件被close                                          |
| IN_CLOSE_NOWRITE | 不可写文件被close                                        |
| IN_OPEN          | 文件被打开                                               |
| IN_MOVE_SELF     | 自移动                                                  |
| IN_MOVED_FROM    | 文件被移走，如mv                                         |
| IN_MOVED_TO      | 文件被移来, 如mv, cp                                     |
| IN_CREATE        | 文件被创建                                               |
| IN_DELETE        | 文件被删除，如rm                                         |
| IN_CLOSE         | 文件被关闭，等同于(IN_CLOSE_WRITE&#124;IN_CLOSE_NOWRITE) |


## 监控日志文件
下面通过一个示例来具体说明pyinotify的用法，我们在```/home/xiaoju/test```目录下面有个```log.txt```文件，现在我们需要监测改文件的变化。
首先我们可以使用```python -m pyinotify -v /home/xiaoju/test```来查看每次文件改动pyinotify的消息。

首先，我们先**touch** log.txt文件，可以看到pyinotify输出了以下的信息
```shell
[2018-07-12 11:26:13,636 pyinotify DEBUG] Event queue size: 128
[2018-07-12 11:26:13,636 pyinotify DEBUG] <_RawEvent cookie=0 mask=0x100 name=log.txt wd=1 >
[2018-07-12 11:26:13,636 pyinotify DEBUG] <_RawEvent cookie=0 mask=0x20 name=log.txt wd=1 >
[2018-07-12 11:26:13,637 pyinotify DEBUG] <_RawEvent cookie=0 mask=0x4 name=log.txt wd=1 >
[2018-07-12 11:26:13,637 pyinotify DEBUG] <_RawEvent cookie=0 mask=0x8 name=log.txt wd=1 >
<Event dir=False mask=0x100 maskname=IN_CREATE name=log.txt path=. pathname=/home/xiaoju/test/log.txt wd=1 >
<Event dir=False mask=0x20 maskname=IN_OPEN name=log.txt path=. pathname=/home/xiaoju/test/log.txt wd=1 >
<Event dir=False mask=0x4 maskname=IN_ATTRIB name=log.txt path=. pathname=/home/xiaoju/test/log.txt wd=1 >
<Event dir=False mask=0x8 maskname=IN_CLOSE_WRITE name=log.txt path=. pathname=/home/xiaoju/test/log.txt wd=1 >
```
总共触发了4个事件,IN_CREATE、IN_OPEN、IN_ATTRIB和IN_CLOSE_WRITE，后面是详细的文件的名称和路径。

 ## 使用python代码监控

In [38]:
import pyinotify
import time
from pyinotify import ProcessEvent,WatchManager,Notifier


log_path = '/home/xiaoju/test'
log_file_name = '%s/log.txt' % log_path
log_file = open(log_file_name, 'r')


class EventHandler(ProcessEvent):
    '''
    定义一个继承ProcessEvent的处理类，处理文件的事件
    '''
    def process_IN_MODIFY(self, event):
        '''处理文件夹发生修改事件，我们需要进行筛选，只处理log.txt的事件
        '''
        if event.name == 'log.txt':
            global log_file
            line = log_file.readline()
            print(line)
    
    def process_IN_CREATE(self, event):
        '''处理文件夹发生文件创建事件，我们需要进行筛选，只处理log.txt的创建事件
        '''
        if event.name == 'log.txt':
            global log_file
            global log_file_name
            print('rotate..')
            log_file.close()
            log_file = open(log_file_name, 'r')  
        
        
wm = WatchManager()
notifier = Notifier(wm, EventHandler())
wm.add_watch(log_path, pyinotify.IN_MODIFY|pyinotify.IN_CREATE)
notifier.loop()

dsds

rotate..
sdsds

